In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import pandas as pd
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset, ClassLabel, Sequence

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ner_dataset_motsunabe.csv")

In [ ]:
# sentence_id 기준으로 문장별 input_ids, labels 리스트 만들기
grouped = df.groupby("sentence_id")
input_ids = grouped["token_id"].apply(list).tolist()
labels = grouped["tag"].apply(list).tolist()

# label → 숫자 인덱스로 변환
unique_labels = sorted(set(l for doc in labels for l in doc))
label2id = {l: i for i, l in enumerate(unique_labels)}
id2label = {i: l for l, i in label2id.items()}
labels_ids = [[label2id[l] for l in doc] for doc in labels]

In [ ]:
# HuggingFace Dataset 생성
dataset = Dataset.from_dict({
    "input_ids": input_ids[:111],
    "labels": labels_ids[:111]
})

train_dataset = dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    "monologg/koelectra-base-v3-discriminator",
    num_labels=len(unique_labels),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.Sch

In [ ]:
args = TrainingArguments(
    output_dir="./ner_kc_electra",
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    metric_for_best_model="eval_loss",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-3351252182.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: psbted (psbted-chung-ang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.520700,1.009487
2,0.917000,0.768692
3,0.734200,0.618527
4,0.598600,0.482638
5,0.479400,0.384534
6,0.387000,0.309104
7,0.329900,0.262612
8,0.280100,0.220876
9,0.245900,0.188745
10,0.212100,0.167719


TrainOutput(global_step=105, training_loss=0.43574830236889067, metrics={'train_runtime': 37.5083, 'train_samples_per_second': 44.39, 'train_steps_per_second': 2.799, 'total_flos': 29501352135378.0, 'train_loss': 0.43574830236889067, 'epoch': 15.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
save_path_drive = "/content/drive/MyDrive/ner_kc_electra_final"

# 모델 및 토크나이저 저장
trainer.save_model(save_path_drive)
tokenizer.save_pretrained(save_path_drive)

('/content/drive/MyDrive/ner_kc_electra_final/tokenizer_config.json',
 '/content/drive/MyDrive/ner_kc_electra_final/special_tokens_map.json',
 '/content/drive/MyDrive/ner_kc_electra_final/vocab.txt',
 '/content/drive/MyDrive/ner_kc_electra_final/added_tokens.json',
 '/content/drive/MyDrive/ner_kc_electra_final/tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
repo_name = "5gidong/kcelectra_review_ner"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...213rk5w/model.safetensors:   0%|          |  570kB /  449MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/5gidong/kcelectra_review_ner/commit/9acb1d46e89e9b84169d6bedec53adf1dac620c2', commit_message='Upload tokenizer', commit_description='', oid='9acb1d46e89e9b84169d6bedec53adf1dac620c2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/5gidong/kcelectra_review_ner', endpoint='https://huggingface.co', repo_type='model', repo_id='5gidong/kcelectra_review_ner'), pr_revision=None, pr_num=None)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("5gidong/kcelectra_review_ner")
tokenizer = AutoTokenizer.from_pretrained("5gidong/kcelectra_review_ner")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/449M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import re

def clean_text(text):
  # 줄바꿈 기준 분리
  sentences = []
  lines = text.split('\n')
  for line in lines:
      line = line.strip()
      if not line:
          continue

      # 느낌표를 마침표로 변환
      bang2dot = re.sub(r'!+', '.', line)
      # 마침표 기준 문장 분리
      sents = re.split(r'(?<=\.)\s+', bang2dot)

      for s in sents:
          s = s.strip()
          if not s:
              continue

          # 한글, 숫자, 공백, 문장부호(., !, ?, ", ', “ ”)만 남기기
          s = re.sub(r'[^가-힣0-9\s\.\?\"\'“”]', '', s)
          if s:
              sentences.append(s)
  return sentences

In [ ]:
import torch

In [ ]:
def get_menu(text):
  sentence = clean_text(text)

  # 4️⃣ 토큰화
  inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

  # 5️⃣ 모델 추론
  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits  # (batch, seq_len, num_labels)
  predictions = torch.argmax(logits, dim=2)

  # 6️⃣ 토큰과 예측 라벨 매핑
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  pred_labels = [id2label[p.item()] for p in predictions[0]]

  # 7️⃣ 결과 출력
  results = []
  temp = []
  concat = False
  for tok, lab in zip(tokens, pred_labels):
      if lab[0] == 'B' and lab[2] == 'M':
        if concat == True:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
        concat = True
        temp.append(tok)

      elif concat:
        if lab[0] == 'I' and lab[2] == 'M':
          temp.append(tok)
        else:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
          concat = False
  return results


In [ ]:
def get_amb(text):
  sentence = clean_text(text)

  # 4️⃣ 토큰화
  inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

  # 5️⃣ 모델 추론
  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits  # (batch, seq_len, num_labels)
  predictions = torch.argmax(logits, dim=2)

  # 6️⃣ 토큰과 예측 라벨 매핑
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  pred_labels = [id2label[p.item()] for p in predictions[0]]

  # 7️⃣ 결과 출력
  results = []
  temp = []
  concat = False
  for tok, lab in zip(tokens, pred_labels):
      if lab[0] == 'B' and lab[2] == 'A':
        if concat == True:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
        concat = True
        temp.append(tok)

      elif concat:
        if lab[0] == 'I' and lab[2] == 'A':
          temp.append(tok)
        else:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
          concat = False
  return results


In [ ]:
def get_exp(text):
  sentence = clean_text(text)

  # 4️⃣ 토큰화
  inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

  # 5️⃣ 모델 추론
  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits  # (batch, seq_len, num_labels)
  predictions = torch.argmax(logits, dim=2)

  # 6️⃣ 토큰과 예측 라벨 매핑
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  pred_labels = [id2label[p.item()] for p in predictions[0]]

  # 7️⃣ 결과 출력
  results = []
  temp = []
  concat = False
  for tok, lab in zip(tokens, pred_labels):
      if lab[0] == 'B' and lab[2] == 'E':
        if concat == True:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
        concat = True
        temp.append(tok)

      elif concat:
        if lab[0] == 'I' and lab[2] == 'E':
          temp.append(tok)
        else:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
          concat = False
  return results


In [ ]:
def get_pp(text):
  sentence = clean_text(text)

  # 4️⃣ 토큰화
  inputs = tokenizer(sentence, return_tensors="pt", truncation=True, is_split_into_words=False)

  # 5️⃣ 모델 추론
  model.eval()
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits  # (batch, seq_len, num_labels)
  predictions = torch.argmax(logits, dim=2)

  # 6️⃣ 토큰과 예측 라벨 매핑
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  pred_labels = [id2label[p.item()] for p in predictions[0]]

  # 7️⃣ 결과 출력
  results = []
  temp = []
  concat = False
  for tok, lab in zip(tokens, pred_labels):
      if lab[0] == 'B' and lab[2] == 'P':
        if concat == True:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
        concat = True
        temp.append(tok)

      elif concat:
        if lab[0] == 'I' and lab[2] == 'P':
          temp.append(tok)
        else:
          results.append(tokenizer.convert_tokens_to_string(temp))
          temp = []
          concat = False
  return results


In [ ]:
sentences = ["아, 매운 명란젓이 코스에 포함되어 있어서 밥을 추가했어요.", "모츠나베를 먹기 전에 소고기 식초 모츠와 냉두부를 선택했습니다.", "센마이도 주문했는데, 개인적으로는 조금 더 크게 썰어주면 좋겠다고 생각했습니다.", "다른 가게보다 조금 비싼데도 불구하고, 거의 젊은 손님들로 가득 차 있었던 것은 데이트에 사용하기 좋기 때문일까요?", "가장 마음에 들었던 것은 챤폰면입니다."]

menu_result = [get_menu(t) for t in sentences]
amb_result = [get_amb(t) for t in sentences]
exp_result = [get_exp(t) for t in sentences]
pp_result = [get_pp(t) for t in sentences]
for idx, sent in enumerate(sentences):
  print("\n", sent)
  print("메뉴: ", menu_result[idx])
  print("분위기: ", amb_result[idx])
  print("경험: ", exp_result[idx])
  print("인원: ", pp_result[idx])


 아, 매운 명란젓이 코스에 포함되어 있어서 밥을 추가했어요.
메뉴:  ['매운 명란젓', '코스', '밥', '추가']
분위기:  []
경험:  []
인원:  []

 모츠나베를 먹기 전에 소고기 식초 모츠와 냉두부를 선택했습니다.
메뉴:  ['모츠나베', '소고기 식초 모츠', '냉두부']
분위기:  []
경험:  ['선택']
인원:  []

 센마이도 주문했는데, 개인적으로는 조금 더 크게 썰어주면 좋겠다고 생각했습니다.
메뉴:  ['센마이도']
분위기:  []
경험:  ['주문', '개인적', '더', '크게', '생각']
인원:  []

 다른 가게보다 조금 비싼데도 불구하고, 거의 젊은 손님들로 가득 차 있었던 것은 데이트에 사용하기 좋기 때문일까요?
메뉴:  []
분위기:  ['데이트']
경험:  ['비싼', '젊은', '손님들']
인원:  []

 가장 마음에 들었던 것은 챤폰면입니다.
메뉴:  ['챤폰면']
분위기:  []
경험:  []
인원:  []
